In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Enable CUDA debugging if needed
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Define the MLP model
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1024 -> 128
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)  # 128 -> 27

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Load your dataset
data = pd.read_csv('EDD_embedding.csv')  # Replace with your file path
X = data.iloc[:, :-1].values  # Features
y = data.iloc[:, -1].values  # Labels

# Check class label range and adjust if necessary
y = y - y.min()  # Ensure labels are in range [0, num_classes - 1]
print("Class labels range after adjustment:", y.min(), y.max())

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocess the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
y_train, y_test = torch.tensor(y_train, dtype=torch.long), torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for batch processing
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32, shuffle=False)

# Instantiate the model, loss function, and optimizer
input_size = 1024
hidden_size = 128
output_size = 27
model = SimpleMLP(input_size, hidden_size, output_size)

# Specify the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to device

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Testing loop
model.eval()
all_labels = []
all_predictions = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to device
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Class labels range after adjustment: 0 26


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
pip install torch pennylane pennylane-qiskit


INFO: pip is looking at multiple versions of pennylane-qiskit to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.8/340.8 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.2

In [ ]:
!pip install semantic_version


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

# Define the quantum device
dev = qml.device('qiskit.aer', wires=4)  # Using 4 qubits

# Define a quantum circuit
@qml.qnode(dev)
def quantum_circuit(params):
    for i in range(4):
        qml.RX(params[i], wires=i)  # Rotations based on input parameters
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]  # Return the expectation values

class HybridModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(HybridModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Only one hidden layer
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size + 4, num_classes)  # Adjust output size for 27 classes

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)

        # Prepare parameters for the quantum circuit
        params = x[:, :4]  # Take first 4 features for quantum circuit

        # Execute quantum circuit for each element in the batch
        quantum_output = []
        for p in params.detach().cpu().numpy():  # Detach the tensor and move it to CPU for numpy conversion
            output = quantum_circuit(p)  # Get the quantum circuit output
            print(f"Quantum circuit output for parameters {p}: {output}")  # Debugging output
            quantum_output.append(output)  # Append the result

        # Flatten quantum_output to ensure it's a 2D array
        quantum_output = np.array(quantum_output)  # Convert to a NumPy array
        quantum_output = quantum_output.reshape(-1, 4)  # Ensure it's 2D, shape (batch_size, 4)

        # Convert quantum output to tensor and ensure it’s on the same device
        quantum_output = torch.tensor(quantum_output, dtype=torch.float32, device=x.device)

        # Concatenate classical and quantum outputs
        x = torch.cat((x, quantum_output), dim=1)  # Combine the outputs
        return self.fc2(x)  # Final classification layer for 27 classes

# Load your dataset
data = pd.read_csv('TGprotein_embed.csv')  # Replace with your file path
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
y = y - y.min()  # Ensure y starts from 0
num_classes = 30  # Set number of classes to 27

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing: Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
y_train, y_test = torch.tensor(y_train, dtype=torch.long), torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for batch processing
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32, shuffle=False)

# Instantiate the model, loss function, and optimizer
input_size = 1024  # Adjust according to your input features
hidden_size = 128  # Single hidden layer size
model = HybridModel(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Specify the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to device

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Testing loop with metrics calculation
model.eval()
all_labels = []
all_predictions = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to device
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate accuracy, precision, recall, and F1 score
accuracy = (torch.tensor(all_predictions) == torch.tensor(all_labels)).sum().item() / len(all_labels)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


/usr/local/lib/python3.10/dist-packages/pennylane/__init__.py:184: PennyLaneDeprecationWarning: QubitDevice will no longer be accessible top level. Please access the class as pennylane.devices.QubitDevice
  warn(


Streaming output truncated to the last 5000 lines.
Quantum circuit output for parameters [6.64042   0.        0.        6.4139256]: [array(0.9453125), array(1.), array(1.), array(0.99609375)]
Quantum circuit output for parameters [0.        3.3155222 0.        0.       ]: [array(1.), array(-0.98046875), array(1.), array(1.)]
Quantum circuit output for parameters [0.         0.07364382 4.0372944  5.915321  ]: [array(1.), array(1.), array(-0.671875), array(0.94335938)]
Quantum circuit output for parameters [0. 0. 0. 0.]: [array(1.), array(1.), array(1.), array(1.)]
Quantum circuit output for parameters [2.3489494  0.         0.97947854 1.3184161 ]: [array(-0.71289062), array(1.), array(0.58203125), array(0.2734375)]
Quantum circuit output for parameters [0.         0.50634396 0.         2.9793785 ]: [array(1.), array(0.87890625), array(1.), array(-0.984375)]
Quantum circuit output for parameters [3.4541733 0.        0.        0.       ]: [array(-0.93945312), array(1.), array(1.), array(1

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

# Define the quantum device
dev = qml.device('qiskit.aer', wires=4)  # Using 4 qubits

# Define a quantum circuit
@qml.qnode(dev)
def quantum_circuit(params):
    for i in range(4):
        qml.RX(params[i], wires=i)  # Rotations based on input parameters
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]  # Return the expectation values

class HybridModel(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super(HybridModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2 + 4, num_classes)  # Ensure input size matches

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)

        # Prepare parameters for the quantum circuit
        params = x[:, :4]  # Take first 4 features for quantum circuit

        # Execute quantum circuit for each element in the batch
        quantum_output = []
        for p in params.detach().cpu().numpy():  # Detach the tensor and move it to CPU for numpy conversion
            output = quantum_circuit(p)  # Get the quantum circuit output
            print(f"Quantum circuit output for parameters {p}: {output}")  # Debugging output
            quantum_output.append(output)  # Append the result

        # Flatten quantum_output to ensure it's a 2D array
        quantum_output = np.array(quantum_output)  # Convert to a NumPy array
        quantum_output = quantum_output.reshape(-1, 4)  # Ensure it's 2D, shape (batch_size, 4)

        # Convert quantum output to tensor and ensure it’s on the same device
        quantum_output = torch.tensor(quantum_output, dtype=torch.float32, device=x.device)

        # Concatenate classical and quantum outputs
        x = torch.cat((x, quantum_output), dim=1)  # Combine the outputs
        return self.fc3(x)  # Final classification layer

# Load your dataset
data = pd.read_csv('DD_dataset.csv')  # Replace with your file path
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
y = y - y.min()  # Ensure y starts from 0
num_classes = len(set(y))

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing: Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
y_train, y_test = torch.tensor(y_train, dtype=torch.long), torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for batch processing
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32, shuffle=False)

# Instantiate the model, loss function, and optimizer
input_size = 1024  # Adjust according to your input features
hidden_size1 = 128
hidden_size2 = 16
model = HybridModel(input_size, hidden_size1, hidden_size2, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Specify the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to device

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Testing loop with metrics calculation
model.eval()
all_labels = []
all_predictions = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to device
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate accuracy, precision, recall, and F1 score
accuracy = (torch.tensor(all_predictions) == torch.tensor(all_labels)).sum().item() / len(all_labels)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Streaming output truncated to the last 5000 lines.
Quantum circuit output for parameters [0.         0.43244693 1.1291474  1.2056909 ]: [array(1.), array(0.9140625), array(0.44140625), array(0.34179688)]
Quantum circuit output for parameters [1.5063124 9.573203  4.138769  3.2260334]: [array(0.04101562), array(-0.9921875), array(-0.5625), array(-1.)]
Quantum circuit output for parameters [1.1236968 0.        3.001965  0.       ]: [array(0.38671875), array(1.), array(-0.99414062), array(1.)]
Quantum circuit output for parameters [0.44160658 0.12141142 5.7713137  2.719269  ]: [array(0.90234375), array(0.9921875), array(0.88085938), array(-0.93554688)]
Quantum circuit output for parameters [3.436353  2.4359539 1.2488872 0.       ]: [array(-0.96679688), array(-0.77539062), array(0.27734375), array(1.)]
Quantum circuit output for parameters [4.3898168 0.        6.4888344 0.       ]: [array(-0.30859375), array(1.), array(0.97265625), array(1.)]
Quantum circuit output for parameters [8.417535 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

# Define the quantum device
dev = qml.device('qiskit.aer', wires=4)  # Using 4 qubits

# Define a quantum circuit
@qml.qnode(dev)
def quantum_circuit(params):
    for i in range(4):
        qml.RX(params[i], wires=i)  # Rotations based on input parameters
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]  # Return the expectation values

class HybridModel(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super(HybridModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2 + 4, num_classes)  # Ensure input size matches

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)

        # Prepare parameters for the quantum circuit
        params = x[:, :4]  # Take first 4 features for quantum circuit

        # Execute quantum circuit for each element in the batch
        quantum_output = []
        for p in params.detach().cpu().numpy():  # Detach the tensor and move it to CPU for numpy conversion
            output = quantum_circuit(p)  # Get the quantum circuit output
            print(f"Quantum circuit output for parameters {p}: {output}")  # Debugging output
            quantum_output.append(output)  # Append the result

        # Flatten quantum_output to ensure it's a 2D array
        quantum_output = np.array(quantum_output)  # Convert to a NumPy array
        quantum_output = quantum_output.reshape(-1, 4)  # Ensure it's 2D, shape (batch_size, 4)

        # Convert quantum output to tensor and ensure it’s on the same device
        quantum_output = torch.tensor(quantum_output, dtype=torch.float32, device=x.device)

        # Concatenate classical and quantum outputs
        x = torch.cat((x, quantum_output), dim=1)  # Combine the outputs
        return self.fc3(x)  # Final classification layer

# Load your dataset
data = pd.read_csv('EDD_embedding.csv')  # Replace with your file path
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
y = y - y.min()  # Ensure y starts from 0
num_classes = len(set(y))

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing: Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
y_train, y_test = torch.tensor(y_train, dtype=torch.long), torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for batch processing
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32, shuffle=False)

# Instantiate the model, loss function, and optimizer
input_size = 1024  # Adjust according to your input features
hidden_size1 = 128
hidden_size2 = 16
model = HybridModel(input_size, hidden_size1, hidden_size2, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Specify the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to device

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Testing loop with metrics calculation
model.eval()
all_labels = []
all_predictions = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to device
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate accuracy, precision, recall, and F1 score
accuracy = (torch.tensor(all_predictions) == torch.tensor(all_labels)).sum().item() / len(all_labels)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Streaming output truncated to the last 5000 lines.
Quantum circuit output for parameters [5.7953014 2.4779024 0.5061482 6.3063807]: [array(0.875), array(-0.78125), array(0.87695312), array(0.99804688)]
Quantum circuit output for parameters [8.972871  8.447562  0.        0.6575825]: [array(-0.921875), array(-0.57226562), array(1.), array(0.78515625)]
Quantum circuit output for parameters [16.065826  12.514446   0.6761389  0.       ]: [array(-0.9453125), array(0.99609375), array(0.76757812), array(1.)]
Quantum circuit output for parameters [ 7.8858476  0.         9.362679  14.824537 ]: [array(-0.06054688), array(1.), array(-1.), array(-0.65234375)]
Quantum circuit output for parameters [6.0443807 2.0906944 6.3914227 0.       ]: [array(0.97070312), array(-0.48632812), array(0.99023438), array(1.)]
Quantum circuit output for parameters [3.4166477 4.7165904 3.4265926 7.5712266]: [array(-0.96289062), array(-0.00195312), array(-0.95703125), array(0.22851562)]
Quantum circuit output for parame

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

# Define the quantum device
dev = qml.device('qiskit.aer', wires=4)  # Using 4 qubits

# Define a quantum circuit
@qml.qnode(dev)
def quantum_circuit(params):
    for i in range(4):
        qml.RX(params[i], wires=i)  # Rotations based on input parameters
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]  # Return the expectation values

class HybridModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(HybridModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Only one hidden layer
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size + 4, num_classes)  # Adjust output size for 27 classes

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)

        # Prepare parameters for the quantum circuit
        params = x[:, :4]  # Take first 4 features for quantum circuit

        # Execute quantum circuit for each element in the batch
        quantum_output = []
        for p in params.detach().cpu().numpy():  # Detach the tensor and move it to CPU for numpy conversion
            output = quantum_circuit(p)  # Get the quantum circuit output
            print(f"Quantum circuit output for parameters {p}: {output}")  # Debugging output
            quantum_output.append(output)  # Append the result

        # Flatten quantum_output to ensure it's a 2D array
        quantum_output = np.array(quantum_output)  # Convert to a NumPy array
        quantum_output = quantum_output.reshape(-1, 4)  # Ensure it's 2D, shape (batch_size, 4)

        # Convert quantum output to tensor and ensure it’s on the same device
        quantum_output = torch.tensor(quantum_output, dtype=torch.float32, device=x.device)

        # Concatenate classical and quantum outputs
        x = torch.cat((x, quantum_output), dim=1)  # Combine the outputs
        return self.fc2(x)  # Final classification layer for 27 classes

# Load your dataset
data = pd.read_csv('EDD_embedding.csv')  # Replace with your file path
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
y = y - y.min()  # Ensure y starts from 0
num_classes = 27  # Set number of classes to 27

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing: Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
y_train, y_test = torch.tensor(y_train, dtype=torch.long), torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for batch processing
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32, shuffle=False)

# Instantiate the model, loss function, and optimizer
input_size = 1024  # Adjust according to your input features
hidden_size = 128  # Single hidden layer size
model = HybridModel(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Specify the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to device

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Testing loop with metrics calculation
model.eval()
all_labels = []
all_predictions = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to device
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate accuracy, precision, recall, and F1 score
accuracy = (torch.tensor(all_predictions) == torch.tensor(all_labels)).sum().item() / len(all_labels)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Streaming output truncated to the last 5000 lines.
Quantum circuit output for parameters [0.         0.40973547 0.         0.        ]: [array(1.), array(0.91796875), array(1.), array(1.)]
Quantum circuit output for parameters [1.5087454 0.        0.        0.       ]: [array(0.0234375), array(1.), array(1.), array(1.)]
Quantum circuit output for parameters [0.         0.83389324 1.388412   0.        ]: [array(1.), array(0.71875), array(0.2109375), array(1.)]
Quantum circuit output for parameters [1.5278507 2.5025125 0.        0.       ]: [array(0.03710938), array(-0.78320312), array(1.), array(1.)]
Quantum circuit output for parameters [0.2549316 0.        2.1230536 2.9233973]: [array(0.98046875), array(1.), array(-0.578125), array(-0.97460938)]
Quantum circuit output for parameters [0.        1.6010631 0.        0.       ]: [array(1.), array(-0.07617188), array(1.), array(1.)]
Quantum circuit output for parameters [0.        0.        1.7856104 3.8168156]: [array(1.), array(1.), arra

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

# Define the quantum device
dev = qml.device('qiskit.aer', wires=4)  # Using 4 qubits

# Define a quantum circuit
@qml.qnode(dev)
def quantum_circuit(params):
    for i in range(4):
        qml.RX(params[i], wires=i)  # Rotations based on input parameters
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]  # Return the expectation values

class HybridModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(HybridModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Only one hidden layer
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size + 4, num_classes)  # Adjust output size for 27 classes

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)

        # Prepare parameters for the quantum circuit
        params = x[:, :4]  # Take first 4 features for quantum circuit

        # Execute quantum circuit for each element in the batch
        quantum_output = []
        for p in params.detach().cpu().numpy():  # Detach the tensor and move it to CPU for numpy conversion
            output = quantum_circuit(p)  # Get the quantum circuit output
            print(f"Quantum circuit output for parameters {p}: {output}")  # Debugging output
            quantum_output.append(output)  # Append the result

        # Flatten quantum_output to ensure it's a 2D array
        quantum_output = np.array(quantum_output)  # Convert to a NumPy array
        quantum_output = quantum_output.reshape(-1, 4)  # Ensure it's 2D, shape (batch_size, 4)

        # Convert quantum output to tensor and ensure it’s on the same device
        quantum_output = torch.tensor(quantum_output, dtype=torch.float32, device=x.device)

        # Concatenate classical and quantum outputs
        x = torch.cat((x, quantum_output), dim=1)  # Combine the outputs
        return self.fc2(x)  # Final classification layer for 27 classes

# Load your dataset
data = pd.read_csv('EDD_embedding.csv')  # Replace with your file path
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
y = y - y.min()  # Ensure y starts from 0
num_classes = 27  # Set number of classes to 27

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing: Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
y_train, y_test = torch.tensor(y_train, dtype=torch.long), torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for batch processing
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32, shuffle=False)

# Instantiate the model, loss function, and optimizer
input_size = 1024  # Adjust according to your input features
hidden_size = 128  # Single hidden layer size
model = HybridModel(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Specify the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
num_epochs = 35
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to device

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Testing loop with metrics calculation
model.eval()
all_labels = []
all_predictions = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to device
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate accuracy, precision, recall, and F1 score
accuracy = (torch.tensor(all_predictions) == torch.tensor(all_labels)).sum().item() / len(all_labels)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


ModuleNotFoundError: No module named 'pennylane'

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load your dataset
data = pd.read_csv('EDD_embedding.csv')  # Replace with your file path
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocess the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the models
mlp = MLPClassifier(hidden_layer_sizes=(128,), max_iter=100, random_state=42)
svm = SVC(probability=True, kernel='linear', random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Create a voting ensemble
ensemble = VotingClassifier(estimators=[
    ('mlp', mlp),
    ('svm', svm),
    ('xgb', xgb)
], voting='soft')  # Use 'soft' voting to consider probabilities

# Train the ensemble model
ensemble.fit(X_train, y_train)

# Make predictions on the test set
y_pred = ensemble.predict(X_test)

# Evaluate the ensemble model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:02:52] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 88.89%
Precision: 0.8950
Recall: 0.8889
F1 Score: 0.8885


In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load your dataset
data = pd.read_csv('EDD_embedding.csv')  # Replace with your file path
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing: Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize classifiers
decision_tree = DecisionTreeClassifier(random_state=42)
naive_bayes = GaussianNB()
knn = KNeighborsClassifier()
svm = SVC(probability=True, random_state=42)  # Using probability=True for voting
log_reg = LogisticRegression(random_state=42)
random_forest = RandomForestClassifier(random_state=42)
xgboost = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Combine classifiers in a voting ensemble
ensemble = VotingClassifier(estimators=[
    ('decision_tree', decision_tree),
    ('naive_bayes', naive_bayes),
    ('knn', knn),
    ('svm', svm),
    ('log_reg', log_reg),
    ('random_forest', random_forest),
    ('xgboost', xgboost)
], voting='soft')  # Using soft voting for probabilities

# Train the ensemble model
ensemble.fit(X_train, y_train)

# Make predictions
y_pred = ensemble.predict(X_test)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:11:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 83.33%
Precision: 0.8430
Recall: 0.8333
F1 Score: 0.8309
